In [9]:
from binance.client import Client
import logging.config
from datetime import datetime
import logging


In [10]:
API_KEY_TEST = 'VdBnQ9rM60MqELvvII4m7Pi6SBk06Urt6AAG2ixMTdX5maagTWrjZlr3MIaHsW9c'
SECRET_KEY_TEST = 'n1owlwcLarcLQNkJgwG8fdrjKgjuuE8aWiCVJcIJygLTPkHGBwPJMVO34yAJMaqK'
API_KEY = "T8OAWzBGVHVJokT6nHxP6Eqo0HNGkl9xgK79TmUXtbQhCI1M2rXwzA4E9C7rSTyI"
SECRET_KEY = "x6BUKF8D9tCEOT0ZyeME0Nchglngudw7HcVD7qzB1uy7BN0c7tUZJGbJDXFrJKD2"
API_TEST_URL = 'https://testnet.binance.vision/api'

In [11]:
class MyAccount:

    INTERVAL_ONE_M = Client.KLINE_INTERVAL_1MINUTE
    INTERVAL_FIVE_M = Client.KLINE_INTERVAL_5MINUTE
    INTERVAL_FIFTEEN_M = Client.KLINE_INTERVAL_15MINUTE
    INTERVAL = [INTERVAL_ONE_M, INTERVAL_FIVE_M, INTERVAL_FIFTEEN_M]
    Client()
    def __init__(self, minimum_order_len, api_key, secret_key, isTest):
        self.client = Client(api_key=api_key, api_secret=secret_key)
        if isTest is True:
            self.client.API_URL = 'https://testnet.binance.vision/api'
        self.account = self.client.get_account()
        self.aceptable_loss = 0.1
        self.max_trade_taxes = 0.001
        self.minimum_order_len = minimum_order_len
        self.bot_status = 'STAND_BY'

    def reverse_list(self, data_list):
        rev = data_list_param[::-1]
        return rev

    def dict_timestamp_to_time(self, data_dict, field_time_name):
        data_dict[field_time_name] = datetime.fromtimestamp(
            data_dict[field_time_name]/1000)
        return data_dict

In [66]:
class Asset(MyAccount):
    def __init__(self, asset_main, asset_pair, minimum_order_len, api_key, secret_key,isTest, interval='5', limit_data=202):
        super().__init__(minimum_order_len, api_key, secret_key,isTest)
        self.asset_main = asset_main.upper()
        self.asset_pair = asset_pair.upper()
        self.symbol = self.asset_main + self.asset_pair
        self.interval = self.INTERVAL_ONE_M
        # self.interval = self.interval[0]
        self.limit_data = limit_data
        # self.asset_candle_data_list = None
        self.tendency = None
        self.actual_price = None
        self.candle_data_list = []
        self.close_price_list = []
        self.close_volume_list = []
        self.my_init_asset_trade_balance = self.get_account_asset_balance(
            self.asset_pair)
        self.current_operated_asset_quantity = None
        self.mm8_now = None
        self.mm8_before = None
        self.mm10_now = None
        self.mm10_before = None
        self.mm20_now = None
        self.mm20_before = None
        self.mm50_now = None
        self.mm50_before = None
        self.mm200_now = None
        self.mm200_before = None
        self.close_now = None
        self.close_before = None
        self.volume_now = None
        self.volume_before = None
        self.data_list_for_dict_print = ['self.mm8_now', 'self.mm8_before',' self.mm10_now', 'self.mm10_before',
                                         'self.mm20_now', 'self.mm20_before', 'self.mm50_now', 'self.mm50_before',                                                    'self.mm200_now', 'self.mm200_before', 'self.close_now', 'self.close_before',                                                'self.volume_now', 'self.volume_before']
        self.data_dict_for_print = {}
        for var in self.data_list_for_dict_print:
            self.data_dict_for_print[var] = eval(var)

    def get_account_asset_balance(self, asset):
        for balance in range(len(self.account['balances'])):
            if self.account['balances'][balance]['asset'] == asset:
                return round(float(self.account['balances'][balance]['free']), 4)

    def get_candle_asset_data(self):
        return self.reverse_list(client.get_klines(symbol=self.symbol, interval=self.interval))

    def mm_now(self, period):
        numbers_series = pd.Series(self.close_price_list)
        windows = numbers_series.rolling(period_number)
        moving_averages = windows.mean()
        moving_averages_list = moving_averages.tolist()
        without_nans = moving_averages_list[period_number - 1:]
        return round(without_nans[0], 3)

    def mm_before(self, period):
        numbers_series = pd.Series(self.close_price_list)
        windows = numbers_series.rolling(period_number)
        moving_averages = windows.mean()
        moving_averages_list = moving_averages.tolist()
        without_nans = moving_averages_list[period_number - 1:]
        return round(without_nans[1], 3)

    def create_close_list(self):
        data = self.get_candle_asset_data()
        for index in range(1, len(data)):
            self.close_price_list.append(float(data[index][4]))
            self.close_volume_list.append(float(data[index][5]))

    def calculate_indicators(self):
        self.create_close_price_list()
        self.create_close_volume_list()
        self.mm8_now = self.mm_now(8)
        self.mm8_before = self.mm_before(8)
        self.mm10_now = self.mm_now(10)
        self.mm10_before = self.mm_before(10)
        self.mm20_now = self.mm_now(20)
        self.mm20_before = self.mm_before(20)
        self.mm50_now = self.mm_now(50)
        self.mm50_before = self.mm_before(50)
        self.mm200_now = self.mm_now(200)
        self.mm200_before = self.mm_before(200)
        self.close_now = self.close_price_list[0]
        self.close_before = self.close_price_list[1]
        self.volume_now = self.close_volume_list[0]
        self.volume_before = self.close_volume_list[1]
        self.actual_price = self.close_now

    def check_asset_tendency_lvl_easy(self):
        if self.mm20_now > self.mm20_before:
            if self.close_now > self.close_before:
                return 'UP'
        if self.mm20_now < self.mm20_before:
            if self.close_now < self.close_before:
                return 'DOWN'
        else:
            return 'STAND'


In [67]:
class Trade(Asset, MyAccount):
    def __init__(self, minimum_order_len, api_key, secret_key, isTest, asset_main, asset_pair, interval='5', limit_data=202):
        super().__init__(asset_main, asset_pair, minimum_order_len, api_key, secret_key, isTest)
        self.order_quantity = None
        self.order_made = None
        self.order_made_id = None
        self.order_made_status = None
        self.order_made_time = None
        self.order_executed = None
        self.order_executed_quantity = None
        self.order_executed_side = None
        self.order_executed_price = None

    def make_market_order(self):
        if self.tendency == 'STAND':
            return
        else:
            self.order_quantity = round(
                self.minimum_order_len/self.actual_price, 3)

            if self.tendency == 'UP':
                self.oder_made = self.client.order_market_buy(
                    symbol=self.symbol, quantity=self.order_quantity)

            if self.tendency == 'DOWN':
                self.oder_made = self.client.order_market_sell(
                    symbol=self.symbol, quantity=self.order_quantity)

            self.order_made = self.dict_timestamp_to_time(
                self.order_made, 'transactTime')
            self.order_made_id = self.order_made['orderId']
            self.order_made_status = self.order_made['status']
            self.order_made_time = self.order_made['transactTime']

    def check_order_made_status(self):
        while True:
            order = self.client.get_order(
                symbol=self.symbol, orderId=self.order_made_id)
            status = order['status']
            if self.order_made_time + timedelta(minutes=5) < datetime.now():
                if status == 'PARTIALLY_FILLED':
                    if self.order_made_time + timedelta(minutes=20) < datetime.now():
                        self.order_made_status = status
                        return
                if status == 'NEW':
                    self.order_made_status = status
                    return
            if status in ['FILLED', 'CANCELED', 'REJECTED', 'EXPIRED']:
                self.order_made_status = status
                return


In [68]:
asset_m = 'BNB'
asset_p = 'USDT'
minimum_order = 0.1
api_k = API_KEY
secret_k = SECRET_KEY
test = False
inter = 5
limit = 202

In [69]:
acc =MyAccount(minimum_order_len=minimum_order, api_key=api_k, secret_key=secret_k, isTest=test)
trade = Trade(asset_main=asset_m, asset_pair=asset_p, minimum_order_len=minimum_order, api_key=api_k, secret_key=secret_k, isTest=test, interval=inter, limit_data=limit)

10

In [1]:
preco_atual = 0.0022327

In [4]:
posicao_atual = 'vendido'

In [5]:
aceito_perder = 0.1

In [6]:
quantidade_bnb = 0.69

In [7]:
quantidade_btc = 0.0

In [15]:
quantidade_ordem_venda_bnb = 0.1
taxas = 0.001

In [35]:
quantidade_btc_pos_venda_bnb = (preco_atual*quantidade_ordem_venda_bnb)
op_taxas = quantidade_btc_pos_venda_bnb*taxas
op_taxas
quantidade_btc_pos_venda_bnb = quantidade_btc_pos_venda_bnb - op_taxas
quantidade_btc_pos_venda_bnb


0.00022304673

In [38]:
novo_preco = 0.005
novo_preco

0.005

In [39]:
qtd_retorno = quantidade_btc_pos_venda_bnb/novo_preco
qtd_retorno 

0.044609346

In [45]:
stop_loss = quantidade_ordem_venda_bnb*aceito_perder
stop_loss

0.010000000000000002

In [46]:
if qtd_retorno < quantidade_ordem_venda_bnb - stop_loss:
    print('loss')

loss
